In [13]:
import importlib
import cv2
import json
import os
import sys

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../')
sys.path.append(NOMEROFF_NET_DIR)

def compare(model_path="../../models/anpr_ocr_box_1-gpu.h5",
           text_detector_name = "box",
           img_format = "png",
           root_dir='/var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val',
           predicted_part_size=1000,
           replace_tamplate = None):
    if replace_tamplate is None:
        replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}
    text_detector_module = importlib.import_module("nomeroff_net.TextDetectors."+text_detector_name)
    text_detector = getattr(text_detector_module, text_detector_name)()
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dir_name, subdir_list, file_list in os.walk(ann_dir):
        for fname in file_list:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []                
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    N = int(len(imgs) / predicted_part_size) + 1
    for i in range(N):
        part = i*predicted_part_size
        part_imgs = imgs[part:part+predicted_part_size]
        predicted_part = text_detector.predict(part_imgs)
        predicted += predicted_part

    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        jsonData["moderation"]["predicted"] = predicted_item
        if jsonData["description"] == jsonData["moderation"]["predicted"]:
            jsonData.update(replace_tamplate)
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}', real: '{}' in file {}".format(jsonData["moderation"]["predicted"],jsonData["description"], json_path))
            err_cnt = err_cnt+1

        with open(json_path, "w") as jsonWF:
            json.dump(jsonData, jsonWF)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [14]:
compare()

LOADED 597 ANNOTATIONS
LOADED 597 IMAGES
PREDICTED 597 IMAGES
Predicted 'H950NSM', real: '2209HC' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/2209HC-0.json
Predicted '13005BAA', real: '13005BAB' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/130BAB05-0.json
Predicted '0350STA', real: '03505STA' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/035STA05-0.json
Predicted '02101DDA', real: '02101DPA' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/021DPA01-0.json
Predicted '08966CXA', real: '08908CXA' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/089CXA08-0.json
Predicted '32402EDA', real: '32402EUA' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/324EUA02-0.json
Predicted 'L089FC', real: '0089FC' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann/0089FC-0.json
Predicted '03905HA', real: '03905CHA' in file /var/www/html2/js/nomeroff-net_2/datasets/ocr/box/val/ann